In [1]:
import pandas as pd
import numpy as np
import csv
import os
import time

In [2]:
from gp_pref_elicit_luisa import gaussian_process as GP 
from gp_pref_elicit_luisa import dataset as data 
from gp_pref_elicit_luisa import acquisition_function as acquisition_function
from gp_pref_elicit_luisa.gp_utilities import utils_ccs as gp_utils_ccs
from gp_pref_elicit_luisa.gp_utilities import utils_data as gp_utils_data
from gp_pref_elicit_luisa.gp_utilities import utils_user as gp_utils_users

In [3]:
# generating synthetic dataset which outputs value vectors
# to generate new dataset: uncomment the line below and comment the dataset above
# gp_utils_ccs.get_ccs(num_objectives=5, ccs_size=20) #feel free to modify the number of objectives
# note: after getting the synthetic data, store the data in a variable and comment the line above otherwise this will 
# not give proper insights for experiments because the data will be generated again.
# synthetic_pcs
# outputs a synthetic Pareto Coverage Set of value vectors with 2 objectives and 100 datapoints

The original dataset, in the data_preprocessed.ipynb file, can not be used for the Gaussian Process as it is not in the form of value vectors. To bring it into the form of value vectors, the dataset needs to be passed into a Multi-Objective Shortest Path Planning Solver which will output value vectors. These value vectors can then be an input for the GP. Thus, for now, I am using a synthetic Pareto Coverage Set (PCS) with objectives ranging from 2 to 7 with 100 datapoints which gives us synthetic value vectors for input to the GP.

In [4]:
synthetic_pcs = np.genfromtxt('synthetic_pcs_sets/obj7size100.csv', delimiter=',')
synthetic_pcs

array([[0.50714808, 0.30882356, 0.39497183, 0.32448755, 0.62139675],
       [0.06553804, 0.33000331, 0.42785094, 0.3675485 , 0.75409167],
       [0.41517771, 0.10088407, 0.72423625, 0.4141903 , 0.34839363],
       [0.31376714, 0.08346241, 0.03128237, 0.93601747, 0.13220026],
       [0.40598955, 0.35697222, 0.60117415, 0.33677081, 0.48261618],
       [0.54843176, 0.00568237, 0.56102165, 0.10392491, 0.61126479],
       [0.37471563, 0.56518861, 0.17317827, 0.4566241 , 0.54923014],
       [0.41034506, 0.12202239, 0.89054853, 0.15284401, 0.01701433],
       [0.43362782, 0.20018085, 0.84249445, 0.06907983, 0.23942768],
       [0.16333289, 0.10467873, 0.39812839, 0.56823598, 0.6935174 ],
       [0.11223828, 0.08108523, 0.79149849, 0.19757381, 0.56153583],
       [0.68340927, 0.14885166, 0.35241843, 0.55810614, 0.27406885],
       [0.52223595, 0.4543887 , 0.08793377, 0.67551948, 0.23820497],
       [0.07724158, 0.41402283, 0.22283937, 0.82549329, 0.3025265 ],
       [0.30030591, 0.30991932, 0.

In [5]:
# initializing GP, Dataset and Acquisition Function 
GP = GP.GPPairwise(num_objectives=7)
utils_comparisons = data.DatasetPairwise(num_objectives=7)
acquisition_function_DA = acquisition_function.DiscreteAcquirer(input_domain=synthetic_pcs, query_type='pairwise', seed=None, acquisition_type='expected improvement')
acquisition_function_EI = acquisition_function.get_expected_improvement(datapoints=synthetic_pcs, gaussian_process=GP, datapoints_hist=acquisition_function_DA.history, xi=0.01)
# getting the user preferences for generating a ground truth utility function
user_pref = gp_utils_users.UserPreference(num_objectives=7, std_noise=0.1)

In [6]:
# ground truth utility function for synthetic dataset
ground_truth_utility_function_dataset = user_pref.get_preference(synthetic_pcs, add_noise=False)

# max utility which is actual best utility
ground_truth_utility_function_dataset = np.max(ground_truth_utility_function_dataset)
ground_truth_utility_function_dataset

0.2924429765278549

In [7]:
# starting points in the dataset
start_points = acquisition_function_DA.get_start_points(gaussian_process=GP) 

# generating ground truth utility function for the starting points
ground_truth_utility_function_start = user_pref.get_preference(start_points, add_noise=False)

# getting the index of the highest utility
highest_utility_index_start = np.argmax(ground_truth_utility_function_start)

# getting the index of the lowest utility
lowest_utility_index_start = np.argmin(ground_truth_utility_function_start)

# mapping the indices of the highest utility to the actual datapoint in the dataset
highest_utility_point_dataset = start_points[highest_utility_index_start]

# mapping the indices of the lowest utility to the actual datapoint in the dataset
lowest_utility_point_dataset = start_points[lowest_utility_index_start]


# adding the highest utility point, which is the winner, to the dataset 
utils_comparisons.add_single_comparison(highest_utility_point_dataset, lowest_utility_point_dataset)

# adding the actual point to the array 
current_max_points = highest_utility_point_dataset

# updating the GP
GP.update(utils_comparisons)

# checking which points to exclude
exclude_points = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)

print('Start points: ', start_points, '\n',
      'Point in the dataset having highest utility: ', highest_utility_point_dataset, '\n',
      'Point in the dataset having lowest utility: ', lowest_utility_point_dataset, '\n',
      'Current max: ', current_max_points, '\n', 
      'Excluded points: ', exclude_points)

Start points:  (array([0.15829472, 0.20786252, 0.44844733, 0.47045835, 0.7136525 ]), array([0.65976021, 0.25484121, 0.1797686 , 0.678351  , 0.08541424])) 
 Point in the dataset having highest utility:  [0.15829472 0.20786252 0.44844733 0.47045835 0.7136525 ] 
 Point in the dataset having lowest utility:  [0.65976021 0.25484121 0.1797686  0.678351   0.08541424] 
 Current max:  [0.15829472 0.20786252 0.44844733 0.47045835 0.7136525 ] 
 Excluded points:  [[0.15829472 0.20786252 0.44844733 0.47045835 0.7136525 ]
 [0.65976021 0.25484121 0.1797686  0.678351   0.08541424]]


We have 2 stopping conditions:
1. If the next point found according to Expected Improvement is already in the list of compared points (the points which we exclude for comparison - excluded points) then the loop breaks. This is because the loop has explored all the points available to it.
2. If we have a large dataset where the above condition might not always be true then we look at the probability of improvement between points. If the probability of improvement is below 5% (0.05) then the loop terminates because there is no further scope of improvement.

In [8]:
# initializing a threshold which is set to 5%
threshold = 0.05
stop = False 
counter = -1 # counter variable to keep track of number of queries
regret_hist = [] # to track and store the regret at each step 
last_counter = 0 # to store the total number of queries (which we get from the last iteration)
last_regret = 0 # to store the regret gained from the last iteration 

first_run = not os.path.exists('experiments/output-EI_7-100.csv') # checking if the file exists already in the directory

regret_file_csv = 'experiments/regret-EI_7-100.csv' 
regret_file_check = os.path.exists(regret_file_csv) # checking if the file containing regret values at each step exists

# we output all values for regret gained at each iteration in a new csv file
with open(regret_file_csv, 'a', newline='') as csvfile:
  write_to_regret_csv = csv.writer(csvfile)

# checking if the file exists already, if not we include the header or else we don't
  if not regret_file_check:
    write_to_regret_csv.writerow(['Step', 'Regret'])

  starting_time = time.time() # starting the timer

  while not stop:
    # next points in the dataset according to Expected Improvement
    next_point_EI = acquisition_function_DA.get_next_point_EI(gaussian_process=GP, exclude=exclude_points) 

    # applying first stopping condition
    if next_point_EI.tolist() in exclude_points.tolist():
      last_counter = counter
      last_regret = regret
      stop = True
      

    # generating ground truth utility function for the next points
    ground_truth_utility_function_next_EI = user_pref.get_preference(next_point_EI, add_noise=False)

    # generating ground truth utility function for current best point
    ground_truth_utility_function_current_best = user_pref.get_preference(current_max_points, add_noise=False)

    # adding the highest utility point, which is the winner, to the dataset only if the current point being evaluated according to 
    # expected improvement has a greater utility than the utility of the current best point evaluated previously
    if ground_truth_utility_function_next_EI > ground_truth_utility_function_current_best:
      utils_comparisons.add_single_comparison(next_point_EI, current_max_points)
      current_max_points = next_point_EI # we update the current best point 
    else:
      utils_comparisons.add_single_comparison(current_max_points, next_point_EI)
    
    # computing the datapoints for this comparisons dataset
    utility_comparisons_datapoints_next_EI = utils_comparisons.datapoints

    # updating the GP
    GP.update(utils_comparisons)

    # checking which points to exclude
    exclude_points = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)
    exclude_points = np.append(exclude_points, [current_max_points], axis=0)

    # difference between actual best utility and observed best utility
    regret = np.subtract(ground_truth_utility_function_dataset, ground_truth_utility_function_current_best)

    # calculating regret for each step
    regret_step = regret
    regret_hist.append(regret_step) # appending the list to contain regret at every step 
    print('Regret at step {}: {}'.format(counter, regret_step))
    write_to_regret_csv.writerow([counter+1, regret_step]) # writing to csv
    print('Regret history saved')

    # calculating the probability of improvement
    prob_imprv = acquisition_function.get_probability_of_improvement(x=next_point_EI, gaussian_process=GP, x_previous=current_max_points)

    # incrementing the counter variable
    counter +=1

    # stop the timer
    ending_time = time.time()
    total_time = ending_time - starting_time

    print('Excluded points: ', exclude_points, '\n',
      'next points for the GP acc to EI: ', next_point_EI, '\n',
      'Comparison dataset datapoints: ', utility_comparisons_datapoints_next_EI, '\n',
      'Current max: ', current_max_points, '\n', 
      'Ground truth utility of the current max: ', ground_truth_utility_function_current_best, '\n',
      'Excluded points: ', len(exclude_points), '\n',
      'Simple Regret: ', regret, '\n',
      'Probability of improvement: ', prob_imprv, '\n',
      'Number of queries: ', counter, '\n',
      'Computation Time: ', total_time)
    
    # applying second stopping condition
    if prob_imprv < threshold:
      last_counter = counter
      last_regret = regret
      stop = True

# we output the counter and regret of the last iteration to a csv for comparisons
with open('experiments/output-EI_7-100.csv', 'a', newline='') as csvfile:
  write_to_csv = csv.writer(csvfile)

  # write the header row only for the first run
  if first_run: # if the file does not exist then we output the header otherwise we output only the data 
    write_to_csv.writerow(['Number of Queries', 'Regret'])
  write_to_csv.writerow([last_counter + 1, last_regret])
print('Successfully written to csv')

Regret at step -1: [0.07061944]
Regret history saved
Excluded points:  [[0.03763238 0.45031401 0.19824097 0.53895945 0.68265974]
 [0.15829472 0.20786252 0.44844733 0.47045835 0.7136525 ]
 [0.03763238 0.45031401 0.19824097 0.53895945 0.68265974]] 
 next points for the GP acc to EI:  [0.03763238 0.45031401 0.19824097 0.53895945 0.68265974] 
 Comparison dataset datapoints:  [[0.15829472 0.20786252 0.44844733 0.47045835 0.7136525 ]
 [0.65976021 0.25484121 0.1797686  0.678351   0.08541424]
 [0.03763238 0.45031401 0.19824097 0.53895945 0.68265974]] 
 Current max:  [0.03763238 0.45031401 0.19824097 0.53895945 0.68265974] 
 Ground truth utility of the current max:  [0.22182354] 
 Excluded points:  3 
 Simple Regret:  [0.07061944] 
 Probability of improvement:  [0.48399068] 
 Number of queries:  0 
 Computation Time:  0.0200653076171875
Regret at step 0: [0.04953861]
Regret history saved
Excluded points:  [[0.03763238 0.45031401 0.19824097 0.53895945 0.68265974]
 [0.15829472 0.20786252 0.448447